In [1]:
# Se importan las librerías que se van a usar
import requests
import time
import re
import pandas as pd
import os
import urllib.parse
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Edge
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from tqdm import tqdm
from urllib.parse import urlparse


# Esta es la web donde aparecen las películas ganadoras y nominadas a mejor película en
# los óscar durante la década de 2010's. La usaremos como punto de partida para
# extraer toda la información
url = "https://www.filmaffinity.com/es/awards3.php?award_id=academy_awards&decade=2010"

# Modificamos el User-Agent para evitar posibles problemas de bloqueo. Se comprueba
# el archivo robots.txt y a priori no deberían bloquearnos
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\
    */*;q=0.8",
    "Accept-Encoding": "gzip, deflate, sdch, br",
    "Accept-Language": "en-US,en;q=0.8",
    "Cache-Control": "no-cache",
    "dnt": "1",
    "Pragma": "no-cache",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36\
     (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36"
}

# URL en la que se hace el login en la web
url_login = "https://www.filmaffinity.com/es/account.ajax.php?action=login"

# Se indican los parámetros que son necesarios para hacer el login
login_data =  {
    'postback':'1',
    'rp':url,
    'username':'Gallardola',
    'password':'P4ssw0rd!'
}

# URL de descarga de imagenes
url_image = 'https://pics.filmaffinity.com/'

# Ruta relativa donde se guardarán las imágenes de las películas
poster_path = "./images_FA"

In [2]:
def get_page(url):
    # Esta función se implementa para extraer el contenido de una URL. Utiliza
    # las cookies de sesión y guarda un tiempo prudencial entre peticiones que
    # es proporcional al tiempo de respuesta del servidor
    try:
        soup = None
        t0 = time.time()
        page = session.get(url, headers=headers)
        delay = time.time() - t0
        if page.status_code == 200:
            soup = BeautifulSoup(page.content)
        else:
            print("Ha habido algún problema al descargar el contenido.\n")
        time.sleep(20*delay)
    except requests.exceptions.Timeout:
        pass
    except requests.exceptions.RequestException:
        pass
    return soup


def find_movie_info(movie_info):
    # Extrae la información que nos interesa de la película y la almacena en
    # una lista
    wanted_data = ["Título original", "Año", "Duración", "País", "Dirección",
                   "Reparto", "Género", "Sinopsis"]
    dt_list = movie_info.find_all("dt")
    dd_list = movie_info.find_all("dd")

    info = [dd_list[i] for i in range(len(dt_list)) if dt_list[i].
            get_text(strip=True) in wanted_data]
    return info


def get_rating_info():
    # Si la página tiene votaciones y una nota la extrae, si no devuelve vacío
    rating_info = []
    result = []
    try:
        rating_info = soup.find(id="rat-avg-container").find_all('div')
        result.append(rating_info[0].get_text(strip=True))
        result.append(rating_info[1].span.get_text(strip=True))
    except AttributeError:
        result = ["", ""]
    return result


def get_reviews_num():
    # Si la pelicula tiene críticas almacena el número total de críticas, si
    # no tiene devuelve el valor 0
    reviews_num = 0
    reviews_info = soup.find("div", id="movie-reviews-box")
    if reviews_info:
        reviews_num = reviews_info.get_text().strip().split()[0]
    return reviews_num


def load_requests(source_url):
    # descarga el cartel o poster de la película, guarda un tiempo prudencial
    # entre descargas proporcional a la respuesta del servidor y retorna la
    # ruta relativa 
    t0 = time.time()
    abs_url = urllib.parse.urljoin(url_image,source_url)
    r = session.get(abs_url, headers=headers, stream=True)
    delay = time.time() - t0
    os.makedirs(poster_path, exist_ok=True)
    if r.status_code == 200:
        ruta = poster_path + source_url
        output = open(ruta, "wb")
        for chunk in r:
            output.write(chunk)
        output.close()
    time.sleep(10*delay)
    return source_url

In [3]:
# Se van a crear una sesión con las credenciales y se van a utilizar para
# acceder a los datos tanto desde la librería requests, como mediante el
# navegador Edge a través de Selenium

# Se abre el navegador Edge
driver = webdriver.Edge()
driver.get(url_login)

# Se crea la sesión para requests
session = requests.Session()

# Se pasan las cookies generadas por el navegador a la sesión requests
for cookie in driver.get_cookies():
    session.cookies.set(cookie["name"], cookie["value"])

# Se hace el login con los datos de usuario
resp = session.post(url_login, data=login_data, headers=headers)
# Se comprueba si el servidor da algún error
for i in resp.history:
    if i.status_code == 401:
        print("Error!")

# Exportamos las cookies de respuesta al navegador
dict_resp_cookies = resp.cookies.get_dict()
response_cookies_browser = [{'name':name, 'value':value} for name,
                            value in dict_resp_cookies.items()]
for c in response_cookies_browser:
    driver.add_cookie(c)

#El navegador ahora contiene las cookies de la autenticación    
driver.get(url_login)

# Se le dan exactamente las mismas cookies a la session de requests que
# las que tiene el navegador para poder tener las 2 sesiones abiertas
for cookie in driver.get_cookies():
    session.cookies.set(cookie['name'], cookie['value'])

In [4]:
# A través de la URL de inicio, se consiguen las URLs para todas las decadas
# en las que se han celebrado los premios Oscars: 1920s a 2020s
decade_soap = get_page(url)
URL_decades = []
decades = decade_soap.find("table", class_='decades').find_all("a")
for i in decades:
    URL_decades.append(i.get("href"))
# Se elimina la última URL que nos conduce a todos los premios
URL_decades.pop()

'https://www.filmaffinity.com/es/all_awards.php'

In [5]:
# Se crea una lista donde se almacenarán las URL de todas las películas
URL_films = []

# Se busca en cada década las URLs de las películas premiadas y nominadas
for dec in tqdm(URL_decades):
    main_soup = get_page(dec)
    # Dentro del div con el atributo class especificado se encuentran las
    # ganadoras del premio
    awarded_films = main_soup.body.find_all(
        "div", class_="aw-mc2 big-poster winner-border")

    # Dentro del div con este atributo class se encuentran las películas
    # que han sido nominadas pero no ganaron el premio
    nominated_films = main_soup.body.find_all(
        "div", class_="aw-mc2 big-poster")

    # Observando su estructura, la URL a la web con toda la información
    # de la película se encuentra dentro de la etiqueta a, en el atributo
    # href, por lo que iteramos para almacenarlas en URL_films
    for i in range(len(awarded_films)):
        URL_films.append(awarded_films[i].a.get("href"))
    for i in range(len(nominated_films)):
        URL_films.append(nominated_films[i].a.get("href"))

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [01:01<00:00,  5.57s/it]


In [6]:
# Se crea un índice que se usara como key del diccionario donde se almacenará
# la información
index = 0
# Se crea el diccionario
data_dict = {}
for i in tqdm(URL_films):
    # Se recorren todas las URLs con las películas buscando la información
    # que se quiere almacenar en el dataset
    soup = get_page(i)
    # Desplegamos los premios ocultos (en caso de que los haya) utilizando
    # Selenium
    driver.get(i)
    try:
        show_hidden_awards = driver.find_element_by_id('show-all-awards')
        time.sleep(5)
        show_hidden_awards.click()
    except NoSuchElementException:
        pass
    except ElementClickInterceptedException:
        # Ejecutamos Javascript (más eficiente) si el delay no es suficiente
        driver.execute_script("arguments[0].click();", show_hidden_awards)

    # Separamos la web en las 2 zonas donde existe información de interés
    movie_info = find_movie_info(soup.find(class_="movie-info"))
    rating_info = get_rating_info()

    # Aquellos atributos que son una lista de valores los generamos previamente
    actor_list = [actor.get_text(strip=True) for actor in movie_info[5].
                  div.find_all("span")[1::2]]
    genre_list = [genre.get_text(strip=True) for genre in movie_info[6].
                  find_all("a")]

    # Almacenamos el voto personal de la película, si el valor es "-1" quiere
    # decir que no se ha visto, por lo que se deja vacío
    my_vote = soup.find("div", class_='rate-movie-box').attrs[
        'data-user-rating']

    # Construimos el diccionario con todos los datos para una película
    data_dict[index] = {
        "ID_FA": re.findall('\\d+', soup.find(
            "meta", property="og:url").get("content"))[0],
        "title": soup.find("h1", id="main-title").span.get_text(strip=True),
        "original_title": movie_info[0].get_text(strip=True),
        "year": movie_info[1].get_text(strip=True),
        "duration": movie_info[2].get_text(strip=True),
        "country": movie_info[3].get_text(strip=True),
        "director": movie_info[4].div.a.get_text(strip=True),
        "actors": actor_list,
        "genre": genre_list,
        "description": movie_info[7].get_text(strip=True),
        "awards": driver.find_element_by_css_selector(
            '.margin-top.movie-info dd').text.split('\n'),
        "average_rating": rating_info[0],
        "rating_votes": rating_info[1],
        "reviews": get_reviews_num(),
        "poster": poster_path + load_requests(urlparse(soup.find(
            id="movie-main-image-container").a.get("href")).path), 
        "my_vote": my_vote if my_vote != "-1" else ""
    }
    index += 1
driver.quit()

100%|██████████████████████████████████████████████████████████████████████████████| 563/563 [1:33:37<00:00,  9.98s/it]


In [7]:
# Se crea un dataframe a partir del diccionario
df = pd.DataFrame().from_dict(data_dict, orient='index')
df.head().append(df.tail())

,ID_FA,title,original_title,year,duration,country,director,actors,genre,description,awards,average_rating,rating_votes,reviews,poster,my_vote
0,602893,La melodía de Broadway,The Broadway Melody,1929,110 min.,Estados Unidos,Harry Beaumont,"[Charles King, Anita Page, Bessie Love, Jed Pr...","[Musical, Romance, Comedia]",Queenie y Hank son dos hermanas que buscan tri...,[1928: Oscar: Mejor película. 3 nominaciones],"5,9",482,10,./images_FA/the_broadway_melody-974223234-larg...,
1,415986,Alas,Wings,1927,138 min.,Estados Unidos,William A. Wellman,"[Clara Bow, Charles 'Buddy' Rogers, Richard Ar...","[Bélico, Acción, Drama, Años 1910-1919, I Guer...",Drama bélico que ha pasado a la historia por s...,[1927: 2 Oscars: Mejor película y Efectos espe...,"7,3",1.363,23,./images_FA/wings-603432841-large.jpg,
2,116361,La coartada (Alibi),Alibi,1929,91 min.,Estados Unidos,Roland West,"[Chester Morris, Harry Stubbs, Mae Busch, Elea...","[Drama, Crimen, Mafia, Policíaco, Melodrama]",Chick Williams desea vengarse de quien lo envi...,[1928: 3 nominaciones al Oscar: Mejor película...,,,0,./images_FA/alibi-628810572-large.jpg,
3,660458,En el viejo Arizona,In Old Arizona,1928,95 min.,Estados Unidos,Irving Cummings,"[Warner Baxter, Edmund Lowe, Dorothy Burgess, ...",[Western],Cisco Kid (Warner Baxter) es un bandido del Vi...,"[1928: 1 Oscar: Mejor actor (Baxter), 5 nom., ...","5,6",52,2,./images_FA/in_old_arizona-995252878-large.jpg,
4,309182,The Hollywood Revue of 1929,The Hollywood Revue of 1929,1929,118 min.,Estados Unidos,Charles Reisner,"[Conrad Nagel, Buster Keaton, Jack Benny, John...",[Musical],Musical por donde irán desfilando los grandes ...,[1929: Oscar: Nominada a la Mejor Película],"5,0",27,0,./images_FA/the_hollywood_revue_of_1929-338377...,
558,845261,Historia de un matrimonio,Marriage Story,2019,136 min.,Estados Unidos,Noah Baumbach,"[Scarlett Johansson, Adam Driver, Laura Dern, ...","[Drama, Drama romántico, Teatro, Familia, Dram...","Charlie, un director de teatro neoyorquino y s...",[2019: Premios Oscar: Mejor actriz secundaria ...,"7,2",27.740,167,./images_FA/marriage_story-714238540-large.jpg,
559,145446,1917,1917,2019,119 min.,Reino Unido,Sam Mendes,"[George MacKay, Dean-Charles Chapman, Mark Str...","[Bélico, Drama, I Guerra Mundial]","En lo más crudo de la Primera Guerra Mundial, ...","[2019: 3 Premios Oscar: Mejor fotografía, soni...","7,8",38.131,373,./images_FA/1917-960418215-large.jpg,
560,169177,Érase una vez en... Hollywood,Once Upon a Time in... Hollywood,2019,165 min.,Estados Unidos,Quentin Tarantino,"[Leonardo DiCaprio, Brad Pitt, Margot Robbie, ...","[Thriller, Drama, Comedia, Años 60, Cine dentr...","Hollywood, años 60. La estrella de un western ...",[2019: Premios Oscar: Mejor actor secundario (...,"7,1",44.793,468,./images_FA/once_upon_a_time_in_hollywood-9871...,7
561,325196,El irlandés,The Irishman,2019,209 min.,Estados Unidos,Martin Scorsese,"[Robert De Niro, Al Pacino, Joe Pesci, Stephen...","[Thriller, Drama, Biográfico, Crimen, Mafia, V...",Frank Sheeran fue un veterano de la Segunda Gu...,"[2019: Premios Oscar: 0/10 nominaciones, inclu...","7,3",31.459,321,./images_FA/the_irishman-973471009-large.jpg,8
562,274299,Le Mans '66,Ford v. Ferrari,2019,152 min.,Estados Unidos,James Mangold,"[Matt Damon, Christian Bale, Jon Bernthal, Cai...","[Drama, Acción, Biográfico, Coches / Automovil...",Se centra en un excéntrico y decidido equipo a...,[2019: 2 Premios Oscar: Mejor montaje y efecto...,"7,1",15.309,101,./images_FA/ford_v_ferrari-929107889-large.jpg,


In [8]:
# Se utiliza el dataframe para exportar la información a CSV
df.to_csv('movie_data.csv', encoding='utf-8-sig')